In [67]:
import pandas as pd
import numpy as np
import re
import hickle as hkl
import torch
import torch.nn.functional as F
from torch.autograd import Variable
from torchviz import make_dot
import os as os
import openml
import json
from autoPyTorch import (AutoNetClassification,
                         AutoNetMultilabel,
                         AutoNetRegression,
                         AutoNetImageClassification,
                         AutoNetImageClassificationMultipleDatasets)
import warnings
warnings.filterwarnings('ignore')

In [68]:
autonet = AutoNetClassification(config_preset="full_cs", result_logger_dir="logs/", 
                                additional_metrics=["accuracy", "auc_metric","pac_metric","balanced_accuracy","cross_entropy"],
                                cross_validator="k_fold", cuda=True
                               )

In [70]:
current_configuration = autonet.get_current_autonet_config()
hyperparameter_search_space = autonet.get_hyperparameter_search_space()
print(hyperparameter_search_space)

Configuration space object:
  Hyperparameters:
    CreateDataLoader:batch_size, Type: UniformInteger, Range: [32, 500], Default: 126, on log-scale
    Imputation:strategy, Type: Categorical, Choices: {mean, median, most_frequent}, Default: mean
    InitializationSelector:initialization_method, Type: Categorical, Choices: {sparse, default}, Default: sparse
    InitializationSelector:initializer:initialize_bias, Type: Categorical, Choices: {Yes, No, Zero}, Default: Yes
    InitializationSelector:sparse:sparsity, Type: Constant, Value: 0.9
    LearningrateSchedulerSelector:adapt:T_max, Type: UniformInteger, Range: [300, 1000], Default: 650
    LearningrateSchedulerSelector:adapt:T_mult, Type: UniformFloat, Range: [1.0, 2.0], Default: 1.5
    LearningrateSchedulerSelector:adapt:patience, Type: UniformInteger, Range: [2, 5], Default: 4
    LearningrateSchedulerSelector:adapt:threshold, Type: UniformFloat, Range: [0.001, 0.5], Default: 0.2505
    LearningrateSchedulerSelector:alternating_cos

In [71]:
autonet.print_help()

Configure AutoNet with the following keyword arguments.
Pass these arguments to either the constructor or fit().

name                                default                  choices                                  type                                     
additional_logs                     []                       []                                       <class 'str'>                            
-----------------------------------------------------------------------------------------------------------------------------------------------
additional_metrics                  [accuracy,               [accuracy,                               <class 'str'>                            
                                     auc_metric,              auc_metric,                                                                      
                                     pac_metric,              pac_metric,                                                                      
                      

In [72]:
task = openml.tasks.get_task(task_id=31)
X, y = task.get_X_and_y()
ind_train, ind_test = task.get_train_test_split_indices()
X_train, Y_train = X[ind_train], y[ind_train]
X_test, Y_test = X[ind_test], y[ind_test]

21:52:33 Data pickle file already exists and is up to date.
21:52:33 Data pickle file already exists and is up to date.


In [73]:
autonet = AutoNetClassification(config_preset="tiny_cs", result_logger_dir="logs/", log_level="info",
                               additional_metrics=["accuracy", "auc_metric","pac_metric","balanced_accuracy"],
                               cross_validator="k_fold", cuda=True)
results_fit = autonet.fit(X_train=X_train,
                          Y_train=Y_train,
                          validation_split=0.3,
                          max_runtime=300,
                          min_budget=60,
                          max_budget=100,
                          refit=True)

with open("logs/results_fit.json", "w") as file:
    json.dump(results_fit, file)

21:52:48 Start autonet with config:
{'embeddings': ['none'], 'lr_scheduler': ['cosine_annealing'], 'networks': ['shapedresnet'], 'preprocessors': ['truncated_svd'], 'target_size_strategies': ['none'], 'over_sampling_methods': ['none'], 'under_sampling_methods': ['none'], 'batch_loss_computation_techniques': ['standard'], 'imputation_strategies': ['median'], 'initialization_methods': ['default'], 'loss_modules': ['cross_entropy_weighted'], 'normalization_strategies': ['standardize'], 'optimizer': ['sgd'], 'hyperparameter_search_space_updates': <autoPyTorch.utils.hyperparameter_search_space_update.HyperparameterSearchSpaceUpdates object at 0x7fc338207410>, 'result_logger_dir': 'logs/', 'log_level': 'info', 'additional_metrics': ['accuracy', 'auc_metric', 'pac_metric', 'balanced_accuracy'], 'cross_validator': 'k_fold', 'cuda': True, 'validation_split': 0.3, 'max_runtime': 300, 'min_budget': 60, 'max_budget': 100, 'categorical_features': None, 'dataset_name': None, 'run_id': '0', 'task_id'

21:52:48 WORKER: start processing job (1, 0, 0)
21:52:48 Fit optimization pipeline
21:52:48 Validation split is set to 0.3 and cross validator specified, autonet will ignore validation split
21:52:48 [Autonet] Continue with cross validation using k_fold
21:52:48 [AutoNet] CV split 0 of 5
21:52:48 Reduced initial budget 19.97668399810791 to cv budget 19.97635374069214 compensate for 0.0016512870788574219
21:52:48 Cannot re-initialize CUDA in forked subprocess. To use CUDA with multiprocessing, you must use the 'spawn' start method
Process pynisher function call:
Traceback (most recent call last):
  File "/home/pollymorphism/miniconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/pollymorphism/miniconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/pollymorphism/miniconda3/lib/python3.7/site-packages/pynisher/limit_function_call.py", line 93, in subprocess_func
   

  File "/home/pollymorphism/miniconda3/lib/python3.7/site-packages/autoPyTorch-0.0.2-py3.7.egg/autoPyTorch/pipeline/nodes/cross_validation.py", line 108, in fit
    result = self.sub_pipeline.fit_pipeline(X=X, Y=Y, **sub_pipeline_kwargs)
  File "/home/pollymorphism/miniconda3/lib/python3.7/site-packages/autoPyTorch-0.0.2-py3.7.egg/autoPyTorch/pipeline/base/pipeline.py", line 60, in fit_pipeline
    return self.root.fit_traverse(**kwargs)
  File "/home/pollymorphism/miniconda3/lib/python3.7/site-packages/autoPyTorch-0.0.2-py3.7.egg/autoPyTorch/pipeline/base/node.py", line 115, in fit_traverse
    node.fit_output = node.fit(**required_kwargs)
  File "/home/pollymorphism/miniconda3/lib/python3.7/site-packages/autoPyTorch-0.0.2-py3.7.egg/autoPyTorch/pipeline/nodes/train_node.py", line 86, in fit
    full_eval_each_epoch=pipeline_config["full_eval_each_epoch"])
  File "/home/pollymorphism/miniconda3/lib/python3.7/site-packages/autoPyTorch-0.0.2-py3.7.egg/autoPyTorch/components/training/trai

  File "/home/pollymorphism/miniconda3/lib/python3.7/site-packages/torch/nn/modules/module.py", line 423, in convert
    return t.to(device, dtype if t.is_floating_point() else None, non_blocking)
  File "/home/pollymorphism/miniconda3/lib/python3.7/site-packages/torch/cuda/__init__.py", line 195, in _lazy_init
    "Cannot re-initialize CUDA in forked subprocess. " + msg)
RuntimeError: Cannot re-initialize CUDA in forked subprocess. To use CUDA with multiprocessing, you must use the 'spawn' start method
21:52:49 Exception occurred using config:
{'CreateDataLoader:batch_size': 125, 'Imputation:strategy': 'median', 'InitializationSelector:initialization_method': 'default', 'InitializationSelector:initializer:initialize_bias': 'No', 'LearningrateSchedulerSelector:lr_scheduler': 'cosine_annealing', 'LossModuleSelector:loss_module': 'cross_entropy_weighted', 'NetworkSelector:network': 'shapedresnet', 'NormalizationStrategySelector:normalization_strategy': 'standardize', 'OptimizerSelector:o

21:52:50 WORKER: registered result for job (4, 0, 0) with dispatcher
21:52:50 job (4, 0, 0) failed with exception
Traceback (most recent call last):
  File "/home/pollymorphism/miniconda3/lib/python3.7/site-packages/hpbandster/core/worker.py", line 206, in start_computation
    result = {'result': self.compute(*args, config_id=id, **kwargs),
  File "/home/pollymorphism/miniconda3/lib/python3.7/site-packages/autoPyTorch-0.0.2-py3.7.egg/autoPyTorch/core/worker.py", line 86, in compute
    raise Exception("Exception in train pipeline. Took " + str((time.time()-start_time)) + " seconds with budget " + str(budget))
Exception: Exception in train pipeline. Took 0.3855321407318115 seconds with budget 100.0

21:52:50 WORKER: start processing job (5, 0, 0)
21:52:50 Fit optimization pipeline
21:52:50 Validation split is set to 0.3 and cross validator specified, autonet will ignore validation split
21:52:50 [Autonet] Continue with cross validation using k_fold
21:52:50 [AutoNet] CV split 0 of 5
21

...

21:56:07 WORKER: registered result for job (436, 0, 0) with dispatcher
21:56:07 job (436, 0, 0) failed with exception
Traceback (most recent call last):
  File "/home/pollymorphism/miniconda3/lib/python3.7/site-packages/hpbandster/core/worker.py", line 206, in start_computation
    result = {'result': self.compute(*args, config_id=id, **kwargs),
  File "/home/pollymorphism/miniconda3/lib/python3.7/site-packages/autoPyTorch-0.0.2-py3.7.egg/autoPyTorch/core/worker.py", line 86, in compute
    raise Exception("Exception in train pipeline. Took " + str((time.time()-start_time)) + " seconds with budget " + str(budget))
Exception: Exception in train pipeline. Took 0.3287668228149414 seconds with budget 100.0

21:56:07 WORKER: start processing job (437, 0, 0)
21:56:07 Fit optimization pipeline
21:56:08 Validation split is set to 0.3 and cross validator specified, autonet will ignore validation split
21:56:08 [Autonet] Continue with cross validation using k_fold
21:56:08 [AutoNet] CV split 0 o

  File "/home/pollymorphism/miniconda3/lib/python3.7/site-packages/autoPyTorch-0.0.2-py3.7.egg/autoPyTorch/core/worker.py", line 124, in optimize_pipeline
    raise e
  File "/home/pollymorphism/miniconda3/lib/python3.7/site-packages/autoPyTorch-0.0.2-py3.7.egg/autoPyTorch/core/worker.py", line 118, in optimize_pipeline
    refit=False, rescore=False, hyperparameter_config_id=config_id, dataset_info=self.dataset_info)
  File "/home/pollymorphism/miniconda3/lib/python3.7/site-packages/autoPyTorch-0.0.2-py3.7.egg/autoPyTorch/pipeline/base/pipeline.py", line 60, in fit_pipeline
    return self.root.fit_traverse(**kwargs)
  File "/home/pollymorphism/miniconda3/lib/python3.7/site-packages/autoPyTorch-0.0.2-py3.7.egg/autoPyTorch/pipeline/base/node.py", line 115, in fit_traverse
    node.fit_output = node.fit(**required_kwargs)
  File "/home/pollymorphism/miniconda3/lib/python3.7/site-packages/autoPyTorch-0.0.2-py3.7.egg/autoPyTorch/pipeline/nodes/cross_validation.py", line 108, in fit
    re

Error parsing results. Check results.json and output for more details. An empty results.json is usually caused by a misconfiguration of AutoNet.


Traceback (most recent call last):
  File "/home/pollymorphism/miniconda3/lib/python3.7/site-packages/autoPyTorch-0.0.2-py3.7.egg/autoPyTorch/pipeline/nodes/optimization_algorithm.py", line 308, in parse_results
    incumbent_trajectory = res.get_incumbent_trajectory(bigger_is_better=False, non_decreasing_budget=False)
  File "/home/pollymorphism/miniconda3/lib/python3.7/site-packages/hpbandster/core/result.py", line 310, in get_incumbent_trajectory
    return_dict['config_ids'].append(return_dict['config_ids'][-1])
IndexError: list index out of range

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/pollymorphism/miniconda3/lib/python3.7/site-packages/autoPyTorch-0.0.2-py3.7.egg/autoPyTorch/pipeline/nodes/optimization_algorithm.py", line 131, in fit
    res = self.parse_results(pipeline_config)
  File "/home/pollymorphism/miniconda3/lib/python3.7/site-packages/autoPyTorch-0.0.2-py3.7.egg/autoPyTorch/pipeline/nodes/op

RuntimeError: No models fit during training, please retry with a larger max_runtime.

In [74]:
autonet_config = {
    "result_logger_dir" : "logs/",
    "budget_type" : "epochs",
    "log_level" : "info", 
    "use_tensorboard_logger" : True,
    "validation_split" : 0.0
    }
autonet = AutoNetClassification(**autonet_config)
hyperparameter_config = autonet.get_hyperparameter_search_space().sample_configuration().get_dictionary()
results_refit = autonet.refit(X_train=X_train,
                              Y_train=Y_train,
                              X_valid=None,
                              Y_valid=None,
                              hyperparameter_config=hyperparameter_config,
                              autonet_config=autonet.get_current_autonet_config(),
                              budget=50)

# Save json
with open("logs/results_refit.json", "w") as file:
    json.dump(results_refit, file)

22:00:05 Start autonet with config:
{'embeddings': ['none'], 'lr_scheduler': ['cosine_annealing', 'plateau'], 'networks': ['shapedresnet'], 'over_sampling_methods': ['smote'], 'preprocessors': ['none', 'truncated_svd', 'power_transformer'], 'target_size_strategies': ['none', 'upsample', 'median'], 'result_logger_dir': 'logs/', 'budget_type': 'epochs', 'log_level': 'info', 'use_tensorboard_logger': True, 'validation_split': 0.0, 'hyperparameter_search_space_updates': None, 'categorical_features': None, 'dataset_name': None, 'run_id': '0', 'task_id': -1, 'algorithm': 'bohb', 'eta': 3, 'min_workers': 1, 'working_dir': '.', 'network_interface_name': 'enp2s0', 'memory_limit_mb': 1000000, 'run_worker_on_master_node': True, 'use_pynisher': True, 'refit_validation_split': 0.0, 'cross_validator': 'none', 'cross_validator_args': {}, 'min_budget_for_cv': 0, 'shuffle': True, 'imputation_strategies': ['mean', 'median', 'most_frequent'], 'normalization_strategies': ['none', 'minmax', 'standardize', 

In [76]:
# See how the random configuration performs (often it just predicts 0)
score = autonet.score(X_test=X_test, Y_test=Y_test)
pred = autonet.predict(X=X_test)

print("Model prediction:", pred[0:10])
print("Accuracy score", score)

Model prediction: [[0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [0.]
 [1.]
 [1.]
 [0.]
 [0.]]
Accuracy score 66.0
<class 'autoPyTorch.core.autonet_classes.autonet_feature_classification.AutoNetClassification'>


In [79]:
pytorch_model = autonet.get_pytorch_model()
print(pytorch_model)

<class 'torch.nn.modules.container.Sequential'>
Sequential(
  (0): Linear(in_features=19, out_features=31, bias=True)
  (1): Sequential(
    (0): ResBlock(
      (shortcut): Linear(in_features=31, out_features=43, bias=True)
      (start_norm): Sequential(
        (0): BatchNorm1d(31, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (1): Tanh()
      )
      (layers): Sequential(
        (0): Linear(in_features=31, out_features=43, bias=True)
        (1): BatchNorm1d(43, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): Tanh()
        (3): Linear(in_features=43, out_features=43, bias=True)
      )
    )
    (1): ResBlock(
      (layers): Sequential(
        (0): BatchNorm1d(43, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (1): Tanh()
        (2): Linear(in_features=43, out_features=43, bias=True)
        (3): BatchNorm1d(43, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (4): Tanh()
  